In [1]:
import cv2
import numpy as np

# Load video
video_path = "241030_16_17_WT_PTZ/241030_16_17_WT_PTZ_Box1_0001.avi"
cap = cv2.VideoCapture(video_path)

# Extract the first frame to detect wells
ret, first_frame = cap.read()
if not ret:
    print("Failed to read video.")
    exit()

# Convert to grayscale
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

# Preprocess the frame to detect the grid
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 150)  # Detect edges

# Detect contours to find wells
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on size (adjust as needed for your wells)
wells = []
for contour in contours:
    area = cv2.contourArea(contour)
    if 500 < area < 5000:  # Adjust thresholds based on well size
        x, y, w, h = cv2.boundingRect(contour)
        wells.append((x, y, w, h))

# Sort wells to align with the 96-well plate grid (manual sorting may be needed)
wells = sorted(wells, key=lambda b: (b[1], b[0]))  # Sort by y, then x coordinates

# Initialize trackers for each well
trackers = [cv2.TrackerCSRT_create() for _ in wells]
for tracker, (x, y, w, h) in zip(trackers, wells):
    tracker.init(first_frame, (x, y, w, h))

# Process video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    for tracker, (x, y, w, h) in zip(trackers, wells):
        success, bbox = tracker.update(frame)
        if success:
            x, y, w, h = map(int, bbox)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw bounding box

    # Display the frame with tracking
    cv2.imshow("Tracking Wells", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [8]:
cv2.destroyAllWindows()

In [ ]:
positions = {i: [] for i in range(len(wells))}  # Dict to store movements
for tracker in trackers:
    success, bbox = tracker.update(frame)
    if success:
        x, y, w, h = map(int, bbox)
        positions[i].append((x, y))